In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
!pip install medmnist albumentations opencv-python matplotlib pillow


In [5]:
!pip install torch torchvision albumentations

In [19]:
!pip install scikit-learn tqdm

In [24]:
# Save this as medmnist_preprocessing.py and run locally

import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import albumentations as A
from medmnist import OCTMNIST, BreastMNIST, PneumoniaMNIST, RetinaMNIST, INFO

# Output directory
os.makedirs('split_data', exist_ok=True)

# Offsets for label uniqueness
offsets = {
    'octmnist': 0,
    'breastmnist': len(INFO['octmnist']['label']),
    'pneumoniamnist': len(INFO['octmnist']['label']) + len(INFO['breastmnist']['label']),
    'retinamnist': len(INFO['octmnist']['label']) + len(INFO['breastmnist']['label']) + len(INFO['pneumoniamnist']['label']),
}

# Preprocessing
def preprocess_general(img):
    if img.ndim == 3 and img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    elif img.ndim == 3 and img.shape[-1] == 1:
        img = img.squeeze()
    return img.astype(np.float32) / 255.0

def preprocess_retina(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    denoised = cv2.medianBlur(gray, 3)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(denoised).astype(np.float32) / 255.0

# Load + preprocess
def load_and_preprocess(dataset_cls, offset, is_retina=False):
    X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []
    for split_name in ['train', 'val', 'test']:
        dataset = dataset_cls(split=split_name, download=True)
        for img, label in zip(dataset.imgs, dataset.labels.squeeze()):
            label += offset
            proc_img = preprocess_retina(img) if is_retina else preprocess_general(img)
            if split_name == 'train':
                X_train.append(proc_img)
                y_train.append(label)
            elif split_name == 'val':
                X_val.append(proc_img)
                y_val.append(label)
            else:
                X_test.append(proc_img)
                y_test.append(label)
    return X_train, y_train, X_val, y_val, X_test, y_test

# Master load
X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []
for name, cls in zip(['octmnist', 'breastmnist', 'pneumoniamnist', 'retinamnist'],
                     [OCTMNIST, BreastMNIST, PneumoniaMNIST, RetinaMNIST]):
    is_retina = (name == 'retinamnist')
    Xt, yt, Xv, yv, Xte, yte = load_and_preprocess(cls, offsets[name], is_retina)
    X_train += Xt
    y_train += yt
    X_val += Xv
    y_val += yv
    X_test += Xte
    y_test += yte

X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

# Print distributions
def print_distribution(name, labels):
    print(f"\n📊 {name} Set Class Distribution:")
    for cls, count in zip(*np.unique(labels, return_counts=True)):
        print(f"Class {cls}: {count}")

print_distribution("Train", y_train)
print_distribution("Validation", y_val)
print_distribution("Test", y_test)

# Augmentation
AUGMENT_CLASSES = [cls for cls, count in zip(*np.unique(y_train, return_counts=True)) if count < 4000]
augment = A.Compose([
    A.Rotate(limit=15, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.3),
    A.ElasticTransform(p=0.2),
])

aug_imgs, aug_labels = [], []
for cls in AUGMENT_CLASSES:
    cls_idxs = np.where(y_train == cls)[0]
    cls_imgs = X_train[cls_idxs]
    needed = 4000 - len(cls_imgs)
    for _ in range(needed):
        img = cls_imgs[np.random.randint(len(cls_imgs))]
        img_aug = augment(image=(img * 255).astype(np.uint8))['image']
        aug_imgs.append(img_aug.astype(np.float32) / 255.0)
        aug_labels.append(cls)

# Append
if aug_imgs:
    X_train = np.concatenate([X_train, np.stack(aug_imgs)])
    y_train = np.concatenate([y_train, np.array(aug_labels)])
    print(f"\n✅ Augmented with {len(aug_labels)} new samples")

# Save
def save_split(X, y, path):
    df = pd.DataFrame(X.reshape((X.shape[0], -1)))
    df['label'] = y
    df.to_csv(path, index=False)

save_split(X_train, y_train, "split_data/train.csv")
save_split(X_val, y_val, "split_data/val.csv")
save_split(X_test, y_test, "split_data/test.csv")

print("\n✅ Saved train, val, and test CSVs to 'split_data/'")


Using downloaded and verified file: /root/.medmnist/octmnist.npz
Using downloaded and verified file: /root/.medmnist/octmnist.npz
Using downloaded and verified file: /root/.medmnist/octmnist.npz
Using downloaded and verified file: /root/.medmnist/breastmnist.npz
Using downloaded and verified file: /root/.medmnist/breastmnist.npz
Using downloaded and verified file: /root/.medmnist/breastmnist.npz
Using downloaded and verified file: /root/.medmnist/pneumoniamnist.npz
Using downloaded and verified file: /root/.medmnist/pneumoniamnist.npz
Using downloaded and verified file: /root/.medmnist/pneumoniamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz
Using downloaded and verified file: /root/.medmnist/retinamnist.npz

📊 Train Set Class Distribution:
Class 0: 33484
Class 1: 10213
Class 2: 7754
Class 3: 46026
Class 4: 147
Class 5: 399
Class 6: 1214
Class 7: 3494
Class 8: 486
Class 9: 128
Class 10: 20

In [25]:
import pandas as pd
import numpy as np

# Load processed splits
df_train = pd.read_csv("split_data/train.csv")
df_val = pd.read_csv("split_data/val.csv")
df_test = pd.read_csv("split_data/test.csv")

def print_class_distribution(df, name):
    labels = df['label'].values
    unique, counts = np.unique(labels, return_counts=True)
    print(f"\n📊 {name} Set Class Distribution:")
    for u, c in zip(unique, counts):
        print(f"Class {u}: {c}")

print_class_distribution(df_train, "Train")
print_class_distribution(df_val, "Validation")
print_class_distribution(df_test, "Test")



📊 Train Set Class Distribution:
Class 0: 33484
Class 1: 10213
Class 2: 7754
Class 3: 46026
Class 4: 4000
Class 5: 4000
Class 6: 4000
Class 7: 4000
Class 8: 4000
Class 9: 4000
Class 10: 4000
Class 11: 4000
Class 12: 4000

📊 Validation Set Class Distribution:
Class 0: 3721
Class 1: 1135
Class 2: 862
Class 3: 5114
Class 4: 21
Class 5: 57
Class 6: 135
Class 7: 389
Class 8: 54
Class 9: 12
Class 10: 28
Class 11: 20
Class 12: 6

📊 Test Set Class Distribution:
Class 0: 250
Class 1: 250
Class 2: 250
Class 3: 250
Class 4: 42
Class 5: 114
Class 6: 234
Class 7: 390
Class 8: 174
Class 9: 46
Class 10: 92
Class 11: 68
Class 12: 20


In [26]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load pretrained ResNet-18
resnet18 = models.resnet18(pretrained=True)

# Modify final FC layer for 13 classes
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 13)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch

class MedDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.X = df.drop(columns=['label']).values.reshape(-1, 1, 28, 28).astype(np.float32)
        self.y = df['label'].values.astype(np.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

# Dataloaders
batch_size = 128

train_loader = DataLoader(MedDataset('split_data/train.csv'), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(MedDataset('split_data/val.csv'), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(MedDataset('split_data/test.csv'), batch_size=batch_size, shuffle=False)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=1e-4)

def evaluate(model, loader):
    model.eval()
    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            X = X.repeat(1, 3, 1, 1)  # Convert 1 channel to 3 channels for ResNet

            outputs = model(X)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())

    # AUC for multi-class
    try:
        auc = roc_auc_score(y_true, y_probs, multi_class='ovr')
    except:
        auc = 0.0

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred, average='weighted'),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='macro'),
        'auc': auc
    }


In [29]:
epochs = 10  # You can tweak this based on val loss
best_val_acc = 0.0

for epoch in range(epochs):
    resnet18.train()
    train_losses = []

    for X, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        X, y = X.to(device), y.to(device)
        X = X.repeat(1, 3, 1, 1)  # 1-channel → 3-channel

        optimizer.zero_grad()
        outputs = resnet18(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    train_metrics = evaluate(resnet18, train_loader)
    val_metrics = evaluate(resnet18, val_loader)

    print(f"\n📊 Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {np.mean(train_losses):.4f}")
    print(f"Train Acc: {train_metrics['accuracy']:.4f} | F1: {train_metrics['f1']:.4f} | Precision: {train_metrics['precision']:.4f} | Recall: {train_metrics['recall']:.4f} | AUC: {train_metrics['auc']:.4f}")
    print(f"Val   Acc: {val_metrics['accuracy']:.4f} | F1: {val_metrics['f1']:.4f} | Precision: {val_metrics['precision']:.4f} | Recall: {val_metrics['recall']:.4f} | AUC: {val_metrics['auc']:.4f}")

    # Save best model
    if val_metrics['accuracy'] > best_val_acc:
        best_val_acc = val_metrics['accuracy']
        torch.save(resnet18.state_dict(), 'best_resnet18.pth')
        print("✅ Saved new best model.")


Epoch 1/10 - Training: 100%|██████████| 1043/1043 [01:05<00:00, 15.98it/s]



📊 Epoch 1/10
Train Loss: 0.5953
Train Acc: 0.8627 | F1: 0.8517 | Precision: 0.8550 | Recall: 0.8627 | AUC: 0.9864
Val   Acc: 0.8819 | F1: 0.8684 | Precision: 0.8716 | Recall: 0.8819 | AUC: 0.9845
✅ Saved new best model.


Epoch 2/10 - Training: 100%|██████████| 1043/1043 [01:05<00:00, 15.98it/s]



📊 Epoch 2/10
Train Loss: 0.3782
Train Acc: 0.8911 | F1: 0.8833 | Precision: 0.8858 | Recall: 0.8911 | AUC: 0.9912
Val   Acc: 0.8884 | F1: 0.8780 | Precision: 0.8789 | Recall: 0.8884 | AUC: 0.9806
✅ Saved new best model.


Epoch 3/10 - Training: 100%|██████████| 1043/1043 [01:05<00:00, 15.99it/s]


...


📊 Epoch 8/10
Train Loss: 0.1286
Train Acc: 0.9618 | F1: 0.9609 | Precision: 0.9616 | Recall: 0.9618 | AUC: 0.9985
Val   Acc: 0.9054 | F1: 0.9014 | Precision: 0.9023 | Recall: 0.9054 | AUC: 0.9655


Epoch 9/10 - Training: 100%|██████████| 1043/1043 [01:05<00:00, 15.99it/s]



📊 Epoch 9/10
Train Loss: 0.1130
Train Acc: 0.9626 | F1: 0.9627 | Precision: 0.9636 | Recall: 0.9626 | AUC: 0.9989
Val   Acc: 0.9093 | F1: 0.9084 | Precision: 0.9095 | Recall: 0.9093 | AUC: 0.9428
✅ Saved new best model.


Epoch 10/10 - Training: 100%|██████████| 1043/1043 [01:04<00:00, 16.05it/s]



📊 Epoch 10/10
Train Loss: 0.1002
Train Acc: 0.9750 | F1: 0.9742 | Precision: 0.9749 | Recall: 0.9750 | AUC: 0.9994
Val   Acc: 0.9136 | F1: 0.9097 | Precision: 0.9106 | Recall: 0.9136 | AUC: 0.9466
✅ Saved new best model.


In [30]:
# Load the best model
resnet18.load_state_dict(torch.load('best_resnet18.pth'))
resnet18.eval()

# Evaluation function with both macro and weighted metrics
def evaluate_full(model, loader):
    model.eval()
    y_true, y_pred, y_probs = [], [], []

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            X = X.repeat(1, 3, 1, 1)

            outputs = model(X)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())

    results = {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted'),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
    }

    try:
        results['auc_ovr'] = roc_auc_score(y_true, y_probs, multi_class='ovr')
    except:
        results['auc_ovr'] = 0.0

    return results

# Evaluate on test set
test_metrics = evaluate_full(resnet18, test_loader)

# Display results
print("\n📊 Final Test Set Evaluation:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")


<ipython-input-30-68da926338f7>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet18.load_state_dict(torch.load('best_resnet18.pth'))



📊 Final Test Set Evaluation:
accuracy: 0.7248
f1_macro: 0.5829
f1_weighted: 0.7097
precision_macro: 0.6167
precision_weighted: 0.7520
recall_macro: 0.5890
recall_weighted: 0.7248
auc_ovr: 0.9279
